In [ ]:
import requests
import re
import os

# CONSTANTS
URL ='https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
URL_PATH = 'https://d37ci6vzurychx.cloudfront.net/trip-data'
OUTPUT_PATH = 'storage/taxi_industry/parquet'
TABLE_NAMES = [
    'yellow_tripdata',
    'green_tripdata',
]


response = requests.get(URL)
download_links = []
for table in TABLE_NAMES:
    # example: href="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet"
    #          href="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet"

    download_links = re.findall(rf'href="({URL_PATH}/.*{table}.*\.parquet)"', response.text)
    download_links.extend(download_links) if download_links else None
print("download_links=",download_links)
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

arr = os.listdir(OUTPUT_PATH)
existing_files = []
for file in arr:
    existing_files.append(file)

for link in download_links:
    filename=link.split('/')[-1]
    if filename in existing_files:
        print(f'Skipping {filename}, already exists.')
        continue
    print(f'Downloading {filename}...')
    response = requests.get(link)
    with open(f'{OUTPUT_PATH}/{filename}', 'wb+') as file:
        file.write(response.content)

print('Finish!')


download_links= ['https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-01.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-02.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-03.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-04.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-05.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-04.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-05.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-06.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-07.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-08.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-09.parquet',